# BA 870 Assignment 4
Cohort A: 

Qiaoling Huang: qiaoling@bu.edu

Shihan Li: shihanli@bu.edu

Ziqin Ma: ziqinm@bu.edu

Chenran Peng: cpeng3@bu.edu

Elmira Ushirova: elu@bu.edu

## Apply two models on given companies

Before running this colab notebook, we have downloaded the required data from WRDS and examined those. To get starter, we uploaded them:

In [1]:
from google.colab import files
uploaded = files.upload()

Saving factors.csv to factors.csv
Saving returns.csv to returns.csv


Then we converted them into pandas data frame and renamed the columns:

In [138]:
import pandas as pd
import numpy as np

# Returns data
df_ret = pd.read_csv('returns.csv')
pd.DataFrame.from_records(df_ret)
df_ret.columns=['PERMNO','Date','Tic','Returns']
df_ret.head()

,PERMNO,Date,Tic,Returns
0,10026,20160129,JJSF,-0.074484
1,10026,20160229,JJSF,0.026023
2,10026,20160331,JJSF,-0.019135
3,10026,20160429,JJSF,-0.066033
4,10026,20160531,JJSF,0.043212


In [139]:
# Factor data

df_fa = pd.read_csv('factors.csv')
pd.DataFrame.from_records(df_fa)
df_fa.columns=['Date','Rm','SMB','HML','Rf']
df_fa.head()

,Date,Rm,SMB,HML,Rf
0,20160129,-0.0577,-0.0335,0.0208,0.0001
1,20160229,-0.0007,0.0079,-0.0050,0.0002
2,20160331,0.0696,0.0087,0.0116,0.0002
3,20160429,0.0092,0.0069,0.0326,0.0001
4,20160531,0.0178,-0.0027,-0.0181,0.0001


For the convinience of calculation, we converted the data type of `Return` column from `object` to `float`:

In [140]:
## Convert 'Return' type from object to float
df_ret['Returns'] = pd.to_numeric(df_ret['Returns'], errors='coerce')
df_ret.dtypes

PERMNO       int64
Date         int64
Tic         object
Returns    float64
dtype: object

Then we checked the missing values in `Returns` data and dropped the rows with NaN in `Returns` column:

In [141]:
## check missing value 
df_ret.isna().sum()
#df.fillna(df.mean())
df_ret.dropna(subset=['Tic', 'Returns'], inplace=True)
## check NA again
df_ret.isna().sum()

PERMNO     0
Date       0
Tic        0
Returns    0
dtype: int64

We removed the `PERMNO` column that will not be used in later steps:

In [0]:
## Remove 'PERMNO' column
df_ret.drop(['PERMNO'], axis=1, inplace=True)

Here we checked the missing values in the `Returns` column again:

In [142]:
df_ret[df_ret['Returns'].isnull()]

,PERMNO,Date,Tic,Returns


Then we cleaned factors data and checked missing values. The result shows that there is no missing.

In [143]:
df_fa.isna().sum() ## there is no missing value

Date    0
Rm      0
SMB     0
HML     0
Rf      0
dtype: int64

Now we calculeted the difference between `Rm` and `Rf`, meanwhile creating a new column in `factors` dataframe to store the values:

In [144]:
df_fa['RmMinusRf'] = df_fa['Rm']-df_fa['Rf']
df_fa.head(60)

,Date,Rm,SMB,HML,Rf,RmMinusRf
0,20160129,-0.0577,-0.0335,0.0208,0.0001,-0.0578
1,20160229,-0.0007,0.0079,-0.0050,0.0002,-0.0009
2,20160331,0.0696,0.0087,0.0116,0.0002,0.0694
3,20160429,0.0092,0.0069,0.0326,0.0001,0.0091
4,20160531,0.0178,-0.0027,-0.0181,0.0001,0.0177
5,20160630,-0.0005,0.0065,-0.0147,0.0002,-0.0007
6,20160729,0.0395,0.0264,-0.0111,0.0002,0.0393
7,20160831,0.0050,0.0116,0.0334,0.0002,0.0048
8,20160930,0.0025,0.0202,-0.0149,0.0002,0.0023
9,20161031,-0.0202,-0.0439,0.0416,0.0002,-0.0204


To substract columns from two data frames, we merged the factor and return data together and did the calculation of `Returns` minus `Rf`:

In [9]:
## merge df_ret and df_fa to a master dataframe
df = pd.merge(df_ret, df_fa, on = 'Date', how = 'left')
df.head(60)

## calculate 'RMinusRf'
df['RMinusRf'] = df['Returns'] - df['Rf']
df.head()

,Date,Tic,Returns,Rm,SMB,HML,Rf,RmMinusRf,RMinusRf
0,20160129,JJSF,-0.074484,-0.0577,-0.0335,0.0208,0.0001,-0.0578,-0.074584
1,20160229,JJSF,0.026023,-0.0007,0.0079,-0.0050,0.0002,-0.0009,0.025823
2,20160331,JJSF,-0.019135,0.0696,0.0087,0.0116,0.0002,0.0694,-0.019335
3,20160429,JJSF,-0.066033,0.0092,0.0069,0.0326,0.0001,0.0091,-0.066133
4,20160531,JJSF,0.043212,0.0178,-0.0027,-0.0181,0.0001,0.0177,0.043112


Then we quickly looked at the summary of the variables:

In [10]:
df.describe()

,Date,Returns,Rm,SMB,HML,Rf,RmMinusRf,RMinusRf
count,6.174100e+04,61741.000000,61741.000000,61741.000000,61741.000000,61741.000000,61741.000000,61741.000000
mean,2.017611e+07,0.014437,0.010915,-0.001091,-0.002638,0.001047,0.009868,0.013390
std,1.118884e+04,0.123662,0.035183,0.023254,0.027232,0.000674,0.035240,0.123687
min,2.016013e+07,-0.912198,-0.095500,-0.047600,-0.049900,0.000100,-0.097400,-0.913998
25%,2.017013e+07,-0.040750,0.001700,-0.020200,-0.020700,0.000300,0.001400,-0.041643
50%,2.018013e+07,0.012461,0.014400,0.001000,-0.010800,0.001100,0.012600,0.011478
75%,2.019013e+07,0.064618,0.031900,0.011500,0.011600,0.001600,0.030400,0.063591
max,2.019123e+07,4.163793,0.084100,0.054700,0.082900,0.002100,0.082000,4.162593


Before running into models, we would like to check those companys with less records, which might not be processed by OLS models. Then we extracted companies with more than 10 months to be our target companies in the models:

In [0]:
tic_select = df['Tic'].value_counts()
tic_select.tail(66)
new_tic = pd.DataFrame(tic_select.head(1391))
tic_ls = list(new_tic.index)

## Estimate CAPM Regression:
*Formula: R-Rf = alpha + Beta*(Rm-Rf) + e

In [86]:
print(tic_ls)

['STZ', 'TAP', 'LEN', 'MKC', 'CBS', 'UA', 'CRTO', 'MORN', 'TTMI', 'NTAP', 'MPB', 'AEE', 'MTD', 'AFL', 'NBL', 'LMAT', 'EL', 'IDXX', 'MKSI', 'SAFM', 'TRV', 'SPNS', 'UDR', 'XOM', 'TXMD', 'BWA', 'TECD', 'UCBI', 'JCS', 'GLW', 'BIDU', 'BRKL', 'PCAR', 'NVMI', 'Z', 'HZNP', 'ICLR', 'ESS', 'CG', 'ARNA', 'GPS', 'FIVN', 'IP', 'DHIL', 'HMST', 'WAT', 'EMR', 'PPC', 'PODD', 'ILMN', 'RVNC', 'ENTG', 'FOXF', 'EIX', 'INTC', 'PCRX', 'ICFI', 'HSIC', 'TISI', 'TEAM', 'OMCL', 'CTRE', 'CORT', 'ADMA', 'FWRD', 'GRPN', 'ERIE', 'IIVI', 'PFE', 'GLPI', 'ACHC', 'CSGS', 'ARCB', 'RPD', 'MIDD', 'INOV', 'MERC', 'KMPR', 'WEC', 'MCK', 'MGM', 'CASH', 'GPN', 'LB', 'TRUP', 'F', 'MCRB', 'CPRT', 'TSCO', 'MOMO', 'FORR', 'TILE', 'ERIC', 'SASR', 'SYBT', 'UMBF', 'HST', 'YNDX', 'WAB', 'ACLS', 'PKI', 'STMP', 'HAIN', 'SIGI', 'SLG', 'ASML', 'BAC', 'BPMC', 'NEOG', 'KO', 'SLGN', 'ENTA', 'TTWO', 'XEL', 'KPTI', 'CMCO', 'ERI', 'M', 'INTU', 'MPWR', 'COKE', 'ACIW', 'DE', 'RYAAY', 'FFIN', 'MHK', 'UNM', 'RGEN', 'PSX', 'STE', 'NWSA', 'TNDM', 'CNS

With the ticker symbols of companies with more than 10 recors in the data frame, we constricued a loop that cam subset each company and run the CAPM Regression sepaately. The output of the loop will be the coefficient of `Rm - Rf`, which is the CAPM_Beta:

In [0]:
import statsmodels.api as sm
capm_df = pd.DataFrame(columns = ['Tic', 'Beta'])

for name in tic_ls :
  mod_df = df.loc[df['Tic'] == name]
  X = mod_df['RmMinusRf']
  y = mod_df['RMinusRf']
  X = sm.add_constant(X)
  model = sm.OLS(y, X).fit()
  capm_df = capm_df.append({'Tic' : name , 'Beta' : model.params[1]} , ignore_index=True)

In [89]:
capm_df

,Tic,Beta
0,STZ,0.787953
1,TAP,0.855555
2,LEN,1.080656
3,MKC,-0.016428
4,CBS,0.934880
...,...,...
1386,GOSS,-0.025796
1387,ALEC,2.445290
1388,TCRR,0.551201
1389,GRSH,0.061971


Then we calculated the average beta of all these tickers:

In [94]:
from statistics import mean

ave_capm_beta = mean(capm_df['Beta'])
print(ave_capm_beta)

1.2136605599699128


Then we printed out the descriptive analysis to see the percentiles of the CAPM betas:

In [95]:
capm_df.describe()

,Beta
count,1391.000000
mean,1.213661
std,0.740942
min,-2.047513
25%,0.757382
50%,1.170017
75%,1.561562
max,6.700845


The mean and medium of the betas are higher than 1, and we can infer that the stocks of these companies are more volatile than the market. They are more sensitive to the market change.

## Estimate Fama-French 3-Factor Model:
R-Rf = alpha + Beta(Rm-Rf) +sSMB + h*HML + e

For the Fama-French model, we modified the previous loop by changing the pre-defined dataframe.

In [0]:
import statsmodels.api as sm
FF_df = pd.DataFrame(columns = ['Tic', 'FF_Beta', 'FF_s', 'FF_h'])

for name in tic_ls :
  mod_df = df.loc[df['Tic'] == name]
  X = mod_df[['RmMinusRf', 'SMB', 'HML']]
  y = mod_df['RMinusRf']
  X = sm.add_constant(X) # adding a constant
  model = sm.OLS(y, X).fit()
  FF_df = FF_df.append({'Tic' : name , 'FF_Beta' : model.params[1], 'FF_s': model.params[2], 'FF_h':model.params[3]} , ignore_index=True)

In [103]:
FF_df

,Tic,FF_Beta,FF_s,FF_h
0,STZ,0.921005,-0.506456,0.093319
1,TAP,1.073410,-0.832880,0.200575
2,LEN,1.103541,-0.083492,-0.031640
3,MKC,0.117872,-0.466685,-0.492508
4,CBS,0.899303,0.109059,0.460978
...,...,...,...,...
1386,GOSS,0.345930,5.552879,-4.655321
1387,ALEC,2.433829,1.638455,-0.936939
1388,TCRR,0.534820,2.378587,-1.361105
1389,GRSH,0.112469,0.240085,0.782763


Next we calculated the average values of three coefficients for all companies listed:

In [104]:
ave_ff_beta = mean(FF_df['FF_Beta'])
print(ave_ff_beta)
ave_ff_s = mean(FF_df['FF_s'])
print(ave_ff_s)
ave_ff_h = mean(FF_df['FF_h'])
print(ave_ff_h)

1.0665516938172608
0.5627390685983974
-0.006081901799550146


Also the descriptive analysis for the coefficients:

In [105]:
FF_df.describe()

,FF_Beta,FF_s,FF_h
count,1391.000000,1391.000000,1391.000000
mean,1.066552,0.562739,-0.006082
std,0.740654,1.313763,1.098802
min,-6.164477,-8.701039,-7.297178
25%,0.638430,-0.070683,-0.475948
50%,1.028555,0.384285,0.016470
75%,1.421269,1.014340,0.561374
max,6.712819,10.340536,7.609706


Compared to the statistics of CAPM_Betas, the mean and median value of FF_Betas are smaller, but still greater than 1. So we can infer that these companies are more sensitive to the market change.

Moreover, FF model involves 2 other factors that takes the weight away from the betas, so the beta can represent more truthful information of that stock.


Finally, we mergered both output dataframes by ticker symbols:

In [107]:
output = pd.merge(capm_df, FF_df, on = "Tic", how = "left")
output

,Tic,Beta,FF_Beta,FF_s,FF_h
0,STZ,0.787953,0.921005,-0.506456,0.093319
1,TAP,0.855555,1.073410,-0.832880,0.200575
2,LEN,1.080656,1.103541,-0.083492,-0.031640
3,MKC,-0.016428,0.117872,-0.466685,-0.492508
4,CBS,0.934880,0.899303,0.109059,0.460978
...,...,...,...,...,...
1386,GOSS,-0.025796,0.345930,5.552879,-4.655321
1387,ALEC,2.445290,2.433829,1.638455,-0.936939
1388,TCRR,0.551201,0.534820,2.378587,-1.361105
1389,GRSH,0.061971,0.112469,0.240085,0.782763


Then we renamed the columns:

In [0]:
output.columns = ['Ticker', 'CAPM_Beta', 'FF-Beta', 'FF-Size', 'FF-Value']

Export the data frame into Excel files:

In [0]:
output.to_excel("Assign4-Output.xlsx")

# Bonus:

In Excel, we combined the YTD-Returns from the given project spreadsheet to our output. Now we uploaded that:

In [116]:
from google.colab import files
uploaded = files.upload()

Saving Assign4-Combine.csv to Assign4-Combine.csv


In [118]:
df_bonus = pd.read_csv('Assign4-Combine.csv')
pd.DataFrame.from_records(df_bonus)
df_bonus.head()

,Unnamed: 0,Ticker,CAPM_Beta,FF-Beta,FF-Size,FF-Value,YTD_Return
0,0,STZ,0.787953,0.921005,-0.506456,0.093319,-0.3035
1,1,TAP,0.855555,1.073410,-0.832880,0.200575,-0.2391
2,2,LEN,1.080656,1.103541,-0.083492,-0.031640,-0.4015
3,3,MKC,-0.016428,0.117872,-0.466685,-0.492508,-0.1346
4,4,CBS,0.934880,0.899303,0.109059,0.460978,-0.5451


## OLS Regression:

Next we ran a regression on `YTD_Return` agains other coefficients to see the correlation.

First, define X and Y:

In [0]:
X = df_bonus[['CAPM_Beta','FF-Beta', 'FF-Size', 'FF-Value']] 
Y = df_bonus['YTD_Return']

Second, fit the X and Y into a OLS regression:

In [0]:
X = sm.add_constant(X) # adding a constant
 
model = sm.OLS(Y, X).fit()

Print out the mdoel summary:

In [131]:
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:             YTD_Return   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     28.49
Date:                Sun, 12 Apr 2020   Prob (F-statistic):           8.91e-23
Time:                        21:33:26   Log-Likelihood:                 32.057
No. Observations:                1391   AIC:                            -54.11
Df Residuals:                    1386   BIC:                            -27.93
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2522      0.012    -20.662      0.0

## Multiple Linear Rergession

Then we tried the multiple regression:

In [0]:
from sklearn import linear_model

lm = linear_model.LinearRegression()
model_lm = lm.fit(X,Y)

Print out the regression R-square:

In [135]:
lm.score(X,Y)

0.07597038745803208

Print out the intercept:

In [136]:
lm.intercept_

-0.2521566205446826

Print out the coefficients:

In [134]:
lm.coef_

array([ 0.        , -0.02661038, -0.02766072, -0.00049677, -0.05132636])

**Explanation:**

As we know, there are 2 types of risks: Systematic risk (market risk) and Idiosyncratic risk (company specific risk). CAPM Beta and FF model factor betas represent the sensitivity of a portfolio’s returns to changes in a systematic factor. In our regressions, we got adjusted R squared of 7%, which means that only 7% of volatility of stocks' returns can be explained by factors associated with market risk. We assume that the rest 93% of variation of stocks' returns can be explained by factors related to the unsystematic or company-related risks. In most examples of company valuations, returns of each company do depend on company's capabilities and other specific factors. 